In [19]:
import pandas as pd
from CommentExtractor import CommentExtractor

vid_lsd = 'JDjTJ6lkb-8'
# vid_tst = 'G25nLDc0o44'

comment_extractor = CommentExtractor()
df = comment_extractor.get_video_comments(vid_lsd)

data = df[["comment-text"]]

In [23]:
print(data.shape)
print(data)

(12718, 1)
                                            comment-text
0               0:01 I'm selling reply button for =1like
1      Sia herhangi bir şarkıda sadece a dese, o şark...
2                                           buen trabajo
3                        Omg this song is killing me !!!
4                                                 Note 9
5      What does You put the running into run mean? I...
6      i would have broken my replay button if it was...
7          The samsung note 9 ad played before this song
8                                       Hay nhất quả đất
9      Obsessed with this song!  Samsung ad brought m...
10     The only good thing Samsung has ever done to m...
11                               8 August is my birthday
12     Lovely song. I really like it especially when ...
13                                       *GALAXY NOTE 9*
14     who else are looking that who got how many lik...
15                                           💕💕💚💚💚😍😍😍😍😍😍
16                  

In [26]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=0.1, max_df=0.9, 
                             stop_words='english',
                             lowercase=True,
                             ngram_range =1 (2, 4))
data_vectorized = vectorizer.fit_transform(data)
print(data_vectorized)


ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.

In [ ]:
# print(data_vectorized)
lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=3, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Non-Negative Matrix Factorization Model
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)
 
# Build a Latent Semantic Indexing Model
lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS)


In [4]:
print(lda_Z[0])
print(nmf_Z[0])
print(lsi_Z[0])

[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


NameError: name 'lsi_Z' is not defined

In [10]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)
 
print("NMF Model:")
print_topics(nmf_model, vectorizer)
print("=" * 20)
 
print("LSI Model:")
print_topics(lsi_model, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('note', 3.676365682880271), ('love', 0.47519594688830913), ('song', 0.36365164162768304), ('sia', 0.35399926682052424), ('samsung', 0.3387148027703936), ('came', 0.3265383874748258)]
Topic 1:
[('sia', 17.027897481054914), ('song', 5.938967317930209), ('love', 3.6437496931255877), ('samsung', 2.912341846385485), ('came', 0.5351233024864767), ('note', 0.5268560731831459)]
Topic 2:
[('samsung', 7.416733749855034), ('love', 7.416623765919542), ('came', 6.60430583573076), ('note', 6.471074722217744), ('sia', 1.6210622606140292), ('song', 0.3655818266852208)]
Topic 3:
[('song', 17.257487473730045), ('came', 3.4826916740538687), ('sia', 1.683638452099731), ('samsung', 0.3737029635110597), ('love', 0.3510317437242871), ('note', 0.3297415519151579)]
Topic 4:
[('came', 1.2373147172996894), ('sia', 1.185141043996546), ('note', 0.40658995175715046), ('love', 0.39012498910373195), ('song', 0.37298055423923865), ('samsung', 0.3676787606442192)]
NMF Model:
Topic 0:
[('sia', 2.84

In [12]:
import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

Loading BokehJS ...

In [13]:
svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(data_vectorized)
 
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(data))
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [14]:
svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(data_vectorized.T)
 
df = pd.DataFrame(columns=['x', 'y', 'word'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorizer.get_feature_names()
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [16]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel


/cern/CommentTopicCorrelation/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      -32.635048  -97.734512       1        1  31.478708
1       15.508331   24.108934       2        1  28.502794
3      144.540405  -97.279900       3        1  22.820961
0       59.634335 -197.549881       4        1   8.922381
4      151.080124   33.948139       5        1   8.275155, topic_info=     Category       Freq     Term      Total  loglift  logprob
term                                                          
2     Default  14.000000     note  14.000000   6.0000   6.0000
5     Default  24.000000     song  24.000000   5.0000   5.0000
4     Default  22.000000      sia  22.000000   4.0000   4.0000
0     Default  14.000000     came  14.000000   3.0000   3.0000
1     Default  13.000000     love  13.000000   2.0000   2.0000
3     Default  12.000000  samsung  12.000000   1.0000   1.0000
3      Topic1   8.043827  samsung  12.567321   0.7097  -1.3940
1      Topic1   8.043707     love  13.521512   0.6365  -1.3940
0      Topic1   7.162707     came  14.368605   0.4597  -1.5100
2      Topic1   7.018211     note  14.833880   0.4075  -1.5304
4      Topic1   1.758125      sia  22.927264  -1.4122  -2.9146
5      Topic1   0.396492     song  24.781418  -2.9793  -4.4040
4      Topic2  16.344744      sia  22.927264   0.9167  -0.5857
1      Topic2   3.497564     love  13.521512  -0.0970  -2.1275
5      Topic2   5.700698     song  24.781418  -0.2143  -1.6390
3      Topic2   2.795500  samsung  12.567321  -0.2479  -2.3516
0      Topic2   0.513654     came  14.368605  -2.0761  -4.0458
2      Topic2   0.505719     note  14.833880  -2.1235  -4.0613
5      Topic3  17.277551     song  24.781418   1.1168  -0.3078
0      Topic3   3.486741     came  14.368605   0.0614  -1.9083
4      Topic3   1.685596      sia  22.927264  -1.1327  -2.6351
3      Topic3   0.374137  samsung  12.567321  -2.0367  -4.1404
1      Topic3   0.351440     love  13.521512  -2.1725  -4.2030
2      Topic3   0.330125     note  14.833880  -2.3277  -4.2655
2      Topic4   6.104653     note  14.833880   1.5287  -0.4091
1      Topic4   0.789069     love  13.521512  -0.4246  -2.4550
3      Topic4   0.562440  samsung  12.567321  -0.6900  -2.7936
0      Topic4   0.542221     came  14.368605  -0.8605  -2.8302
4      Topic4   0.587820      sia  22.927264  -1.2471  -2.7495
5      Topic4   0.603848     song  24.781418  -1.2979  -2.7226
0      Topic5   2.663281     came  14.368605   0.8064  -1.1633
4      Topic5   2.550979      sia  22.927264   0.2961  -1.2063
3      Topic5   0.791417  samsung  12.567321  -0.2731  -2.3767
1      Topic5   0.839732     love  13.521512  -0.2870  -2.3175
2      Topic5   0.875172     note  14.833880  -0.3383  -2.2762
5      Topic5   0.802829     song  24.781418  -0.9378  -2.3624, token_table=      Topic      Freq     Term
term                          
0         1  0.487173     came
0         2  0.069596     came
0         3  0.208789     came
0         4  0.069596     came
0         5  0.208789     came
1         1  0.591650     love
1         2  0.221869     love
1         4  0.073956     love
1         5  0.073956     love
2         1  0.471893     note
2         2  0.067413     note
2         4  0.404479     note
2         5  0.067413     note
3         1  0.636572  samsung
3         2  0.238714  samsung
3         4  0.079571  samsung
3         5  0.079571  samsung
4         1  0.087232      sia
4         2  0.697859      sia
4         3  0.087232      sia
4         4  0.043616      sia
4         5  0.130849      sia
5         2  0.242117     song
5         3  0.685998     song
5         4  0.040353     song
5         5  0.040353     song, R=6, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 4, 1, 5])